### Generating Global embedding

In [1]:
import numpy as np
import networkx as nx

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

from gensim.models import Word2Vec

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import sklearn.linear_model as LinearModels

import seaborn as sns
import matplotlib.pyplot as plt

import os
import pandas as pd
from tqdm import *
from scipy import stats

from pathlib import Path

NathanPath="d:\Documents\Info\INF554\INF554_Kaggle_Project"
NathanPath="/users/eleves-a/2019/nathan.peluso/INF554/INF554_Kaggle_Project"

project_path = str(Path(os.getcwd()).parent.absolute())
print("Current directory : " + os.getcwd() + ", Project directory : " + project_path)

os.chdir(project_path)
os.chdir(NathanPath)

Current directory : /users/eleves-a/2019/nathan.peluso/INF554/INF554_Kaggle_Project/Global, Project directory : /users/eleves-a/2019/nathan.peluso/INF554/INF554_Kaggle_Project


In [56]:
vectors=np.load("Global/paper_vectors.npy")

In [57]:
id_abstracts=vectors[:,0].astype(np.int64)

In [58]:
print(len(id_abstracts))
print(np.max(id_abstracts))
print(np.min(id_abstracts))

624181
2908511069
3603


In [59]:
num_id_abstracts=dict([(a,b) for a,b in enumerate(id_abstracts)])
id_abstracts_num=dict([(b,a) for a,b in enumerate(id_abstracts)])

In [65]:
len(id_abstracts_num.keys())

624181

In [60]:
with open("data/author_papers.txt") as f:
    authors_papers=f.readlines()

In [61]:
n_dim=vectors.shape[1]-1
authors_vectors=np.zeros((len(authors_papers), n_dim+1), dtype=np.float64)

In [68]:
papers=authors_papers[0].split("\n")[0].split(":")[1].split("-")
print(papers)
p=papers[0]
print(int(p))
print(id_abstracts_num.get(int(p)))

['1510273386', '1827736641', '1588673897', '2252711322', '2123653597']
1510273386
58046


In [78]:
s=0
for i,author in enumerate(authors_papers):
    papers=author.split("\n")[0].split(":")[1].split("-")
    vector=np.zeros(n_dim)
    no_fail=False
    for p in papers:
        try:
            vector+=vectors[id_abstracts_num[int(p)], 1:]
            no_fail=True
        except KeyError:
            pass
    if (not no_fail):
        s+=1
    authors_vectors[i][0]=int(author.split(":")[0])
    if (np.linalg.norm(vector)>0):
        vector=vector/np.linalg.norm(vector)
    authors_vectors[i][1:]=vector.copy()
print(s)


1880


In [79]:
np.save("Global/authors_vectors.npy", authors_vectors)

## Concatenate

In [93]:
embeddings_improved=np.load("DeepWalk/embeddings_improved.npy")
authors_vectors=np.load("Global/authors_vectors.npy")
G = nx.read_edgelist('data/coauthorship.edgelist', delimiter=' ', nodetype=int)

In [94]:
print(embeddings_improved.shape)
print(authors_vectors.shape)

num_id_author=dict([(a,b) for a,b in enumerate(authors_vectors[:,0])])
id_author_num=dict([(b,a) for a,b in enumerate(authors_vectors[:,0])])

num_nodeID=dict([(a,b) for a,b in enumerate(G.nodes)])
node_ID_num=dict([(b,a) for a,b in enumerate(G.nodes)])

(217801, 133)
(217801, 151)


In [96]:
n_lines=len(G.nodes)
n_dim_tot=133+151
n_emb=embeddings_improved.shape[1]
n_abs=authors_vectors.shape[1]-1
full_matrix=np.zeros((n_lines, n_dim_tot), dtype=np.float64)
for i in range(n_lines):
    node=num_nodeID[i]
    full_matrix[i,0]=node
    full_matrix[i,1:1+n_emb]=embeddings_improved[i].copy()
    full_matrix[i,1+n_emb:]=authors_vectors[id_author_num[node],1:].copy()



In [99]:
np.save("Global/full_embedding_matrix.npy", full_matrix)

In [100]:
full_matrix.shape

(217801, 284)

## MLP

## MLP Submission
